In [1]:
from urllib.parse import urljoin
from urllib.parse import urlparse
from urllib.request import urlopen
from bs4 import BeautifulSoup
import validators
import requests
import warnings
import threading

warnings.filterwarnings('ignore')

In [2]:
def get_full_link(url: str, base_url: str) -> str:
    domain = urlparse(base_url).netloc
    if url:
        if not url.startswith("http"):  
            url = urljoin(base_url, url)
            if validators.url(url):
                return url
        elif domain not in url:              
            return
        else:
            if validators.url(url):
                return url

In [3]:
lock = threading.Lock()

def get_links(url: str, base_url: str) -> list[str]:
    if url:
        try:
            response = requests.get(url, verify=False)
            soup = BeautifulSoup(response.content, "html.parser", from_encoding="iso-8859-1")
        except Exception as error:
            print(error)
            return
        links = []
        for element in soup.find_all(name=['a', 'link']):
            link = element.get("href")
            link = get_full_link(link, base_url)
            if link:
                links.append(link)
        return links

In [4]:
def worker(url: str, chank_size: int) -> None:
    global checked
    global not_checked
    global in_work

    while not_checked or in_work>0:
        if not not_checked:
            continue

        in_work += 1

        with lock:
            chank = []
            for _ in range(chank_size):
                if not_checked == []:
                    break
                chank.append(not_checked.pop(0))

        for link in chank:
            if link:
                link = get_full_link(link, url)
                new_links = get_links(link, url)
                if new_links:
                    for i in new_links:
                        if not i in not_checked and not i in checked:
                            not_checked.append(i)
                if link not in checked:
                    checked.append(link)   

        in_work -= 1   


In [9]:
def crawler(site: str, chank_size: int) -> None:
    threads = []

    for n in range(4):
        t = threading.Thread(target=worker, args=(site, chank_size))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

    with open(f'{urlparse(site).netloc}.txt', 'w', encoding="utf-8") as fp:
        for line in checked:
            if line:
                fp.write( line + "\n")

In [10]:
site = "http://crawler-test.com/"
checked = [site]
not_checked = get_links(site, site)
in_work = 0
chank_size = 20
crawler(site, chank_size)


Exceeded 30 redirects.
Exceeded 30 redirects.
HTTPSConnectionPool(host='invalid.crawler-test.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001F1B5568DF0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Exceeded 30 redirects.
Exceeded 30 redirects.
HTTPSConnectionPool(host='crawler-test.com', port=80): Max retries exceeded with url: /canonical_tags/canonical_port_80 (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:997)')))
HTTPSConnectionPool(host='crawler-test.com', port=8080): Max retries exceeded with url: /canonical_tags/canonical_port_8080 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001F1BA8EE0B0>, 'Connection to crawler-test.com timed out. (connect timeout=None)'))
